## **Assignment 6: Tugas Phyton 2**

Nama: Muh. Adrian Hidayat
NIM : 1102200515

## **1.) 75% training, 25% testing**

## Import library yang digunakan

In [656]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import make_scorer

Memasukkan data yang akan diolah

In [657]:
from google.colab import drive
drive.mount('/content/drive') #meminta izin untuk mengakses drive kemudian memountnya ke path yang ada
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CPA1-3/CPA1.csv') # aktifkan untuk membaca file CPA1 kemudian menyimpannya kedalam dataframe variabel df
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CPA1-3/CPA2.csv') # aktifkan untuk membaca file CPA2 kemudian menyimpannya kedalam dataframe variabel df
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CPA1-3/CPA3.csv') # aktifkan untuk membaca file CPA3 kemudian menyimpannya kedalam dataframe variabel df
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CPC1-5/CPC1.csv') # aktifkan untuk membaca file CPC1 kemudian menyimpannya kedalam dataframe variabel df
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CPC1-5/CPC2.csv') # aktifkan untuk membaca file CPC2 kemudian menyimpannya kedalam dataframe variabel df
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CPC1-5/CPC3.csv') # aktifkan untuk membaca file CPC3 kemudian menyimpannya kedalam dataframe variabel df
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CPC1-5/CPC4.csv') # aktifkan untuk membaca file CPC4 kemudian menyimpannya kedalam dataframe variabel df
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CPC1-5/CPC5.csv') # aktifkan untuk membaca file CPC5 kemudian menyimpannya kedalam dataframe variabel df
label_encoder = LabelEncoder() #membuat objek label encoder berfungsi menyimpan label menjadi nilai numerik
df['kelas'] = label_encoder.fit_transform(df['kelas']) #mentransform nilai kelas pada dataframe sesuai dengan model fit dari label encoder untuk mengubah nilai kelas menjadi nilai numerik
print(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
       powLv1    powLv2    powLv3    powLv4    powLv5  kelas
0    0.446692  0.220117  0.144672  0.106003  0.082516      0
1    0.442455  0.219658  0.145039  0.107494  0.085354      0
2    0.446993  0.220535  0.144443  0.105565  0.082465      0
3    0.457090  0.221898  0.141954  0.101666  0.077392      0
4    0.447407  0.220674  0.143812  0.105975  0.082134      0
..        ...       ...       ...       ...       ...    ...
495  0.444264  0.220282  0.144946  0.106833  0.083676      2
496  0.445045  0.220472  0.144977  0.106466  0.083040      2
497  0.445374  0.220198  0.145034  0.106326  0.083069      2
498  0.449798  0.221450  0.143373  0.105009  0.080371      2
499  0.446266  0.220902  0.144428  0.105523  0.082881      2

[500 rows x 6 columns]


##Preprocessing Data

In [658]:

X = df.iloc[:, :-1]  # penentuan fitur (mengambambil semua kolom kecuali kolom terakhir) atau dapat menggunakan syntax x= df.drop(['label'], index = 0 (baris)/ 1 (kolom))
y = df.iloc[:, -1]   # Target ( mengambil kolom target/ label yaitu kolom terakhir)  atau dapat menggunakan syntax y=df ['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

##preprocessing data untuk SVM
scaler = StandardScaler() # membuat objek dari kelas StandardScaler yang akan digunakan untuk menormalkan atau menskalakan fitur-fitur dataset.
X_train_scaled = scaler.fit_transform(X_train)  #untuk menormalkan fitur-fitur dalam data training
X_test_scaled = scaler.transform(X_test) #menerapkan transformasi yang sama yang diterapkan pada data training ke data testing

##Proses klasifikasi SVM

In [659]:
svm_classifier = SVC() #pembuatan objek klasifikasi SVM melalui library SVC
svm_param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear']} ##penentuan nilai C, gamma, dan kernel yang mana yang berpengaruh pada akurasi adalah kernel
svm_grid_search = GridSearchCV(svm_classifier, svm_param_grid, cv=10, scoring='accuracy') #membuat objek GridSearchCV guna mencari best parameter untuk SVM, dimana CV=10 berarti melakukan validasi silang sebanyak 10 kali dan menjadikan akurasi sebagai metrik valuasi
svm_grid_search.fit(X_train_scaled, y_train) # melakukan fit pada grid hingga menemukan best parameter dari data
svm_best_params = svm_grid_search.best_params_

svm_classifier = SVC(**svm_best_params) #membuat objek klasifikasi SVM lagi namun dengan parameter terbaik
svm_classifier.fit(X_train_scaled, y_train) #melakukan fit dengan model SVM terbaru

svm_predictions = svm_classifier.predict(X_test_scaled) #melakukan prediksi menggunakan model SVM yang telah dilatih pada data test yang telah dinormalisasi

svm_conf_matrix = confusion_matrix(y_test, svm_predictions) # menghitung matriks kebingungan (confusion matrix) antara nilai target sebenarnya (y_test) dan nilai prediksi dari model SVM.
print(confusion_matrix(y_test, svm_predictions))
#svm_TN, svm_FP, svm_FN, svm_TP = svm_conf_matrix.ravel() #diperlukan untuk mengekstrak nilai conf matriks (TP,TN,FP,FN) untuk digunakan dalam proses evaluasi model (specificity, dll). kemudian penggunaan metode ravel untuk mengubah conf matriks dari 2x2 menjadi 1 dimensi.
svm_TN, svm_FP, svm_FN, svm_TP, _, _, _, _, _ = svm_conf_matrix.ravel() #dinyalakan jika matriksnya berukuran 3x3


[[33  9  9]
 [ 7 37  3]
 [15  5  7]]


## Proses klasifikasi KNN

In [660]:
knn_classifier = KNeighborsClassifier() #membuat objek klasifikasi KNN

knn_param_grid = {'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance']} #mendefinisikan grid parameter yang akan dijelajahi dalan gridsearch mulai dari  nilai tetangga dan juga bobot
knn_grid_search = GridSearchCV(knn_classifier, knn_param_grid, cv=10, scoring='accuracy') #membuat objek GridSearchCV guna mencari best parameter untuk KNN, dimana CV=10 berarti melakukan validasi silang sebanyak 10 kali dan menjadikan akurasi sebagai metrik valuasi
knn_grid_search.fit(X_train_scaled, y_train) # melakukan fit pada grid hingga menemukan best parameter dari data
knn_best_params = knn_grid_search.best_params_

knn_classifier = KNeighborsClassifier(**knn_best_params)  #membuat objek klasifikasi KNN lagi namun dengan parameter terbaik
knn_classifier.fit(X_train_scaled, y_train)  #melakukan fit dengan model KNN terbaru

knn_predictions = knn_classifier.predict(X_test_scaled) #melakukan prediksi menggunakan model SVM yang telah dilatih pada data test yang telah dinormalisasi

knn_conf_matrix = confusion_matrix(y_test, knn_predictions) # menghitung matriks kebingungan (confusion matrix) antara nilai target sebenarnya (y_test) dan nilai prediksi dari model SVM.
print (knn_conf_matrix)
#knn_TN, knn_FP, knn_FN, knn_TP = knn_conf_matrix.ravel() #diperlukan untuk mengekstrak nilai conf matriks (TP,TN,FP,FN) untuk digunakan dalam proses evaluasi model (specificity, dll). kemudian penggunaan metode ravel untuk mengubah conf matriks dari 2x2 menjadi 1 dimensi.
knn_TN, knn_FP, knn_FN, knn_TP, _, _, _, _, _ = knn_conf_matrix.ravel() #dinyalakan jika matriksnya berukuran 3x3

print (knn_conf_matrix)

[[38  8  5]
 [ 6 34  7]
 [16  3  8]]
[[38  8  5]
 [ 6 34  7]
 [16  3  8]]


## Hasil

Melihat hasil confusion matrix dari model SVM dan KNN

In [661]:
print("Confusion Matrix svm :")
print(confusion_matrix(y_test, svm_predictions))

svm_TN = confusion_matrix(y_test, svm_predictions)[0, 0]
svm_FP = confusion_matrix(y_test, svm_predictions)[0, 1]
svm_FN = confusion_matrix(y_test, svm_predictions)[1, 0]
svm_TP = confusion_matrix(y_test, svm_predictions)[1, 1]

print("Confusion Matrix knn :")
print(confusion_matrix(y_test, knn_predictions))

knn_TN = confusion_matrix(y_test, knn_predictions)[0, 0]
knn_FP = confusion_matrix(y_test, knn_predictions)[0, 1]
knn_FN = confusion_matrix(y_test, knn_predictions)[1, 0]
knn_TP = confusion_matrix(y_test, knn_predictions)[1, 1]

Confusion Matrix svm :
[[33  9  9]
 [ 7 37  3]
 [15  5  7]]
Confusion Matrix knn :
[[38  8  5]
 [ 6 34  7]
 [16  3  8]]


 Evaluasi model berdasarkan nilai dari 4 parameter yang ingin diuji ( Akurasi, Sensitivitas, Specifitas, F1-Score )

In [662]:
#Evaluasi model
svm_predictions = svm_classifier.predict(X_test_scaled)  #mencari nilai prediksi dari data validasi yang telah dinormalisasi menggunakan model SVM yang telah dilatih
knn_predictions = knn_classifier.predict(X_test_scaled)  #mencari nilai prediksi dari data validasi yang telah dinormalisasi menggunakan model KNN yang telah dilatih

#mencari nilai akurasi dari model SVM dan KNN
svm_accuracy = accuracy_score(y_test, svm_predictions)
knn_accuracy = accuracy_score(y_test, knn_predictions)

#membuat report dari model SVM dan KNN
svm_report = classification_report(y_test, svm_predictions, target_names=label_encoder.classes_)
knn_report = classification_report(y_test, knn_predictions, target_names=label_encoder.classes_)

#mencari nilai sensitivity dari model SVM dan KNN
#svm_sensitivity = svm_TP / (svm_TP + svm_FN)
#knn_sensitivity = knn_TP / (knn_TP + knn_FN)
svm_recall = recall_score(y_test, svm_predictions, average='weighted')
knn_recall = recall_score(y_test, knn_predictions, average='weighted')

#mencari nilai specificity dari model SVM dan KNN
svm_specificity = svm_TN / (svm_TN + svm_FP)
knn_specificity = knn_TN / (knn_TN + knn_FP)

#mencari nilai f1 score dari model SVM dan KNN
svm_f1 = f1_score(y_test, svm_predictions, average='weighted')
knn_f1 = f1_score(y_test, knn_predictions, average='weighted')


Print Hasil Akurasi, Sensitivitas, Specifitas, F1-Score

In [663]:
print('Sensitivity klasifikasi SVM: ', svm_recall)
print("Specificity kalsifikasi SVM:", svm_specificity)
print("Akurasi Klasifikasi SVM:", svm_accuracy)
print("F1-Score Klasifikasi SVM:", svm_f1)
print("\nLaporan Klasifikasi SVM:")
print(svm_report)

print("Sensitivity klasifikasi K-NN:", knn_recall)
print("Specificity klasifikasi K-NN:", knn_specificity)
print("Akurasi Klasifikasi K-NN:", knn_accuracy)
print("F1-Score Klasifikasi K-NN:", knn_f1)
print("\nLaporan Klasifikasi K-NN:")
print(knn_report)

Sensitivity klasifikasi SVM:  0.616
Specificity kalsifikasi SVM: 0.7857142857142857
Akurasi Klasifikasi SVM: 0.616
F1-Score Klasifikasi SVM: 0.6036952336307779

Laporan Klasifikasi SVM:
              precision    recall  f1-score   support

           A       0.60      0.65      0.62        51
           C       0.73      0.79      0.76        47
           E       0.37      0.26      0.30        27

    accuracy                           0.62       125
   macro avg       0.56      0.56      0.56       125
weighted avg       0.60      0.62      0.60       125

Sensitivity klasifikasi K-NN: 0.64
Specificity klasifikasi K-NN: 0.8260869565217391
Akurasi Klasifikasi K-NN: 0.64
F1-Score Klasifikasi K-NN: 0.6307963097232292

Laporan Klasifikasi K-NN:
              precision    recall  f1-score   support

           A       0.63      0.75      0.68        51
           C       0.76      0.72      0.74        47
           E       0.40      0.30      0.34        27

    accuracy               

## **2.) 50% training, 25% validation, dan 25% testing**

## Preprocessing Data

In [664]:
X = df.iloc[:, :-1]  # penentuan fitur (mengambambil semua kolom kecuali kolom terakhir) atau dapat menggunakan syntax x= df.drop(['label'], index = 0 (baris)/ 1 (kolom))
y = df.iloc[:, -1]   # Target ( mengambil kolom target/ label yaitu kolom terakhir)  atau dapat menggunakan syntax y=df ['label']

X_temp, X_train, y_temp, y_train = train_test_split(X, y, test_size=0.5, random_state=42)

X_test, X_val, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) #digunakan untuk membagi data temp menjadi data train dan validasi

##preprocessing data untuk SVM
scaler = StandardScaler() # membuat objek dari kelas StandardScaler yang akan digunakan untuk menormalkan atau menskalakan fitur-fitur dataset.
X_train_scaled = scaler.fit_transform(X_train)  #untuk menormalkan fitur-fitur dalam data training
X_val_scaled = scaler.transform(X_val) #menerapkan transformasi yang sama yang diterapkan pada data training ke data validasi
X_test_scaled = scaler.transform(X_test) #menerapkan transformasi yang sama yang diterapkan pada data training ke data testing


## Hasil

 Evaluasi model berdasarkan nilai dari 4 parameter yang ingin diuji ( Akurasi, Sensitivitas, Specifitas, F1-Score )

In [665]:
#Evaluasi model
svm_predictions = svm_classifier.predict(X_test_scaled)  #mencari nilai prediksi dari data validasi yang telah dinormalisasi menggunakan model SVM yang telah dilatih
knn_predictions = knn_classifier.predict(X_test_scaled)  #mencari nilai prediksi dari data validasi yang telah dinormalisasi menggunakan model KNN yang telah dilatih

#mencari nilai akurasi dari model SVM dan KNN
svm_accuracy = accuracy_score(y_val, svm_predictions)
knn_accuracy = accuracy_score(y_val, knn_predictions)

#membuat report dari model SVM dan KNN
svm_report = classification_report(y_val, svm_predictions, target_names=label_encoder.classes_)
knn_report = classification_report(y_val, knn_predictions, target_names=label_encoder.classes_)

#mencari nilai sensitivity dari model SVM dan KNN
#svm_sensitivity = svm_TP / (svm_TP + svm_FN)
#knn_sensitivity = knn_TP / (knn_TP + knn_FN)
svm_recall = recall_score(y_val, svm_predictions, average='weighted')
knn_recall = recall_score(y_val, knn_predictions, average='weighted')

#mencari nilai specificity dari model SVM dan KNN
svm_specificity = svm_TN / (svm_TN + svm_FP)
knn_specificity = knn_TN / (knn_TN + knn_FP)

#mencari nilai f1 score dari model SVM dan KNN
svm_f1 = f1_score(y_val, svm_predictions, average='weighted')
knn_f1 = f1_score(y_val, knn_predictions, average='weighted')



Print Hasil Akurasi, Sensitivitas, Specifitas, F1-Score

In [666]:
print("Sensitivity Klasifikasi SVM:", svm_recall)
print("Specificity Klasifikasi SVM:", svm_specificity)
print("Akurasi Klasifikasi SVM:", svm_accuracy)
print("F1-Score Klasifikasi SVM:", svm_f1)
print("\nLaporan Klasifikasi SVM:")
print(svm_report)


print("Sensitivity Klasifikasi K-NN:", knn_recall)
print("Specificity Klasifikasi K-NN:", knn_specificity)
print("Akurasi Klasifikasi K-NN:", knn_accuracy)
print("F1-Score Klasifikasi K-NN:", knn_f1)
print("\nLaporan Klasifikasi K-NN:")
print(knn_report)

Sensitivity Klasifikasi SVM: 0.792
Specificity Klasifikasi SVM: 0.7857142857142857
Akurasi Klasifikasi SVM: 0.792
F1-Score Klasifikasi SVM: 0.7761797776573385

Laporan Klasifikasi SVM:
              precision    recall  f1-score   support

           A       0.75      0.83      0.79        48
           C       0.84      0.96      0.90        50
           E       0.73      0.41      0.52        27

    accuracy                           0.79       125
   macro avg       0.78      0.73      0.74       125
weighted avg       0.79      0.79      0.78       125

Sensitivity Klasifikasi K-NN: 0.768
Specificity Klasifikasi K-NN: 0.8260869565217391
Akurasi Klasifikasi K-NN: 0.768
F1-Score Klasifikasi K-NN: 0.7597276466108148

Laporan Klasifikasi K-NN:
              precision    recall  f1-score   support

           A       0.75      0.83      0.79        48
           C       0.80      0.86      0.83        50
           E       0.72      0.48      0.58        27

    accuracy              

## **3.) 10 Fold CV**

## Preprocessing Data

In [667]:
# Inisialisasi model SVM dan K-NN menggunakan parameter terbaik yang telah ditentukan sebelumnnya
svm_classifier = SVC(**svm_best_params)
knn_classifier = KNeighborsClassifier(**knn_best_params)

# Inisialisasi skalar untuk penskalaan fitur
scaler = StandardScaler()

# Penskalaan fitur
X_scaled = scaler.fit_transform(X)

## Perhitungan score validasi silang

In [668]:
kf = KFold(n_splits=10, shuffle=True, random_state=42) #membuat objek kf yang berisikan kfold dari library scikit learn dengan jumlahsplit atau validasi silang sebanyak 10 bagian dan 10 kali uji. dimana data akan diacak (shuffle true)

def specificity_score(y_true, y_pred):
    svm_TN, svm_FP, svm_FN, svm_TP, _, _, _, _, _ = confusion_matrix(y_true, y_pred).ravel() #nyalakan jika bentuk confusion matrixnya 3x3
    #svm_TN, svm_FP, svm_FN, svm_TP = confusion_matrix(y_true, y_pred).ravel()
    return svm_TN / (svm_TN + svm_FP)

def specificity_score1(y_true, y_pred):
    knn_TN, knn_FP, knn_FN, knn_TP, _, _, _, _, _ = confusion_matrix(y_true, y_pred).ravel() #nyalakan jika bentuk confusion matrixnya 3x3
    #knn_TN, knn_FP, knn_FN, knn_TP = confusion_matrix(y_true, y_pred).ravel()
    return knn_TN / (knn_TN + knn_FP)

    #penentuan posisi nilai confusion matriks yang dibutuhkan, digunakan untuk mencari posisi TN,TP,FN,FP juga harus digunakan jika conf matriks menghasilkan matriks 2x2
    svm_TN = confusion_matrix(y_test, svm_predictions)[0, 0]
    svm_FP = confusion_matrix(y_test, svm_predictions)[0, 1]
    svm_FN = confusion_matrix(y_test, svm_predictions)[1, 0]
    svm_TP = confusion_matrix(y_test, svm_predictions)[1, 1]

    knn_TN = confusion_matrix(y_test, knn_predictions)[0, 0]
    knn_FP = confusion_matrix(y_test, knn_predictions)[0, 1]
    knn_FN = confusion_matrix(y_test, knn_predictions)[1, 0]
    knn_TP = confusion_matrix(y_test, knn_predictions)[1, 1]

#SVM
akurasi_svm = cross_val_score(svm_classifier, X_scaled, y, cv = kf, scoring='accuracy')
print('Cross-validation akurasi SVM:{}'.format(akurasi_svm))
f1_score_svm = cross_val_score(svm_classifier, X_scaled, y, cv = kf, scoring= 'f1_macro')
print('Cross-validation f1-score SVM:{}'.format(f1_score_svm))
sensitivity_svm = cross_val_score(svm_classifier, X_scaled, y, cv = kf, scoring= 'recall_macro')
print('Cross-validation sensitivity SVM:{}'.format(sensitivity_svm))
specificity_svm = cross_val_score(svm_classifier, X_scaled, y, cv = kf, scoring= make_scorer(specificity_score))
print('Cross-validation specificity SVM:{}'.format(specificity_svm))

#KNN
akurasi_knn = cross_val_score(knn_classifier, X_scaled, y, cv = kf, scoring='accuracy')
print('Cross-validation akurasi KNN:{}'.format(akurasi_knn))
f1_score_knn = cross_val_score(knn_classifier, X_scaled, y, cv = kf, scoring= 'f1_macro')
print('Cross-validation f1-score KNN:{}'.format(f1_score_knn))
sensitivity_knn = cross_val_score(knn_classifier, X_scaled, y, cv = kf, scoring='recall_macro')
print('Cross-validation sensitivity KNN:{}'.format(sensitivity_knn))
specificity_knn = cross_val_score(knn_classifier, X_scaled, y, cv = kf, scoring= make_scorer(specificity_score1))
print('Cross-validation specificity KNN:{}'.format(specificity_knn))



Cross-validation akurasi SVM:[0.64 0.58 0.76 0.74 0.82 0.76 0.68 0.78 0.66 0.76]
Cross-validation f1-score SVM:[0.62294686 0.51128357 0.65365642 0.72004608 0.76296296 0.69831441
 0.5531401  0.74065207 0.56013986 0.7117763 ]
Cross-validation sensitivity SVM:[0.62214452 0.52376396 0.65175983 0.74592075 0.76258235 0.70196581
 0.57045455 0.73333333 0.57449495 0.70572391]
Cross-validation specificity SVM:[0.88235294 0.63157895 1.         0.81818182 0.9        0.90909091
 0.7        0.83333333 0.76923077 0.80952381]
Cross-validation akurasi KNN:[0.6  0.58 0.84 0.8  0.88 0.78 0.72 0.7  0.72 0.82]
Cross-validation f1-score KNN:[0.56146973 0.53412698 0.72621871 0.78244631 0.8442029  0.75648352
 0.62652885 0.63573274 0.69433107 0.79769722]
Cross-validation sensitivity KNN:[0.56379176 0.53564593 0.7184265  0.78904429 0.80737813 0.76076923
 0.62424242 0.64444444 0.68989899 0.78754209]
Cross-validation specificity KNN:[0.88888889 0.68421053 1.         0.91304348 0.90909091 0.9
 0.77272727 0.8333333

## Hasil 10 fold CV

In [669]:
#SVM
print('Hasil dari 10 fold CV untuk klasifikasi dengan model SVM sebagai berikut:')
print('Average cross-validation sensitivity: {:.2f}'.format(sensitivity_svm.mean()))
print('Average cross-validation specificity: {:.2f}'.format(specificity_svm.mean()))
print('Average cross-validation akurasi: {:.2f}'.format(akurasi_svm.mean()))
print('Average cross-validation f1-score: {:.2f}'.format(f1_score_svm.mean()))


#KNN
print('Hasil dari 10 fold CV untuk klasifikasi dengan model KNN sebagai berikut:')
print('Average cross-validation sensitivity: {:.2f}'.format(sensitivity_knn.mean()))
print('Average cross-validation specificity: {:.2f}'.format(specificity_knn.mean()))
print('Average cross-validation akurasi: {:.2f}'.format(akurasi_knn.mean()))
print('Average cross-validation f1-score: {:.2f}'.format(f1_score_knn.mean()))


Hasil dari 10 fold CV untuk klasifikasi dengan model SVM sebagai berikut:
Average cross-validation sensitivity: 0.66
Average cross-validation specificity: 0.83
Average cross-validation akurasi: 0.72
Average cross-validation f1-score: 0.65
Hasil dari 10 fold CV untuk klasifikasi dengan model KNN sebagai berikut:
Average cross-validation sensitivity: 0.69
Average cross-validation specificity: 0.85
Average cross-validation akurasi: 0.74
Average cross-validation f1-score: 0.70
